In [9]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "./tujutuju-1e76f2c20811.json"

import pickle
import argparse
import numpy as np
import googleapiclient.discovery
from google.api_core.client_options import ClientOptions
from tensorflow.keras.preprocessing.sequence import pad_sequences

def token_and_pad(input_string, max_length=120, trunc_type='post', padding_type='post'):
  sentences = [input_string]
  
  f = open('./tokenizer.pickle', 'rb')
  tokenizer = pickle.load(f)

  tokenizer.fit_on_texts(sentences)
  word_index = tokenizer.word_index

  sequences = tokenizer.texts_to_sequences(sentences)
  padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

  return padded.tolist()

def predict_json(project, region, model, instances, version=None):
    """Send json data to a deployed model for prediction.

    Args:
        project (str): project where the Cloud ML Engine Model is deployed.
        region (str): regional endpoint to use; set to None for ml.googleapis.com
        model (str): model name.
        instances ([Mapping[str: Any]]): Keys should be the names of Tensors
            your deployed model expects as inputs. Values should be datatypes
            convertible to Tensors, or (potentially nested) lists of datatypes
            convertible to tensors.
        version: str, version of the model to target.
    Returns:
        Mapping[str: any]: dictionary of prediction results defined by the
            model.
    """
    # Create the ML Engine service object.
    # To authenticate set the environment variable
    # GOOGLE_APPLICATION_CREDENTIALS=<path_to_service_account_file>
    prefix = "{}-ml".format(region) if region else "ml"
    api_endpoint = "https://{}.googleapis.com".format(prefix)
    client_options = ClientOptions(api_endpoint=api_endpoint)
    service = googleapiclient.discovery.build(
        'ml', 'v1', client_options=client_options)
    name = 'projects/{}/models/{}'.format(project, model)

    if version is not None:
        name += '/versions/{}'.format(version)

    response = service.projects().predict(
        name=name,
        body={'instances': instances}
    ).execute()

    if 'error' in response:
        raise RuntimeError(response['error'])

    classes = ["beach", "hotel", "museum", "restaurant", "temple"]
    cls_index = np.argmax(response['predictions'][0], axis=-1)
    cls = classes[cls_index]

    return cls

def parse_args():
    parser = argparse.ArgumentParser(description="Prediction")
    parser.add_argument("--input", required=True, help='Text Input (String)')
    return parser.parse_args()

if __name__ == "__main__":
    args = parse_args()
    inst = args.input
    predict_json("tujutuju", "asia-southeast1", "capstone", inst)

In [10]:
input_string = "beach"
print(predict_json("tujutuju", "asia-southeast1", "capstone", token_and_pad(input_string)))

beach


In [8]:
import numpy as np
res = predict_json("tujutuju", "asia-southeast1", "capstone", token_and_pad(input_string))
np.argmax(res[0], axis=-1)

0